# 卷積神經網路
CNN使用於影像辨識、聲音辨識等各種情況。在視覺辨識競賽中，深度學習使用的手法幾乎都是以CNN為基礎。本章將詳細說明CNN的結構，並且利用Python執行處理內容。
前面說明過的神經網路是，相鄰各層的所有神經元彼此相連，稱為**全連接( _fully connected_ )**，我們以Affine層的名稱，執行過全連接層。如圖所示，利用Affine層，可以建構出5層全連接的神經網路。<br>
![Cnn](./img/ch7-CNN.PNG)<br><br>

那用CNN會形成何種結構呢?如下圖所示<br>
![Cnn_full](./img/ch7.cnn_fully.PNG)<br>
CNN加入新的「Convolution層」與「polling層」。CNN各層的連接順序是「Convlution - ReLU - (Polling)」(有時會省略Polling層)。並於最後輸出層，使用「Affine - ReLU」組合，再將結果以「Affine-Softmax」執行特徵歸一化(全連接層)的動作，這是一般常見的CNN結構。


## 7.2 卷積層
在CNN中，出現了padding、stride等CNN專用的名詞。另外，傳遞在各層的資料變成具有形狀的資料(例如，三維資料)，與之前的全連接網路不同。因此，我們在這裡花多點時間，徹底介紹CNN所使用的卷積層結構。


### 7.2.1 全連接層的問題
前面介紹的全連接神經網路，使用了全連接層( Affine層 )。全連接層連接相鄰各層的所有神經元，可以決定任意輸出數量。<br><br>

全連接層的問題在於，他會「忽略」資料的形狀。例如，輸入資料為影像，影像通常是有水平、垂直、色板方向的三維形狀。可是，輸入全連接層時，三維資料必須變成平面(一維)。以前面提過的MNIST資料集為例子，輸入影像就是(1,28,28)，1色板、垂直28像素、水平28像素的形狀，全連接層卻會把這個形狀變成一行784(28x28)個資料，做為輸入資料，輸入最初的Affine層。<br><br>

然而，卷積層(Convolution層)能維持形狀。如果是影像，輸入資料可以當作三維資料來處理，對上下一層輸出同樣是三維的資料。因此，CNN(可能)可以正確了解影像等含有形狀的資料。<br><br>

有時，我們會把CNN的卷積層輸出入的資料稱作**輸入特徵圖(_input feature map_)**，與**輸出特徵圖(_output feature map_)**，而本書將上面「輸出特徵圖」之專有名詞與「特徵圖」視為等意詞。


### 7.2.2 卷積運算
![卷積運算步驟](./img/7.2.2.PNG)<br>
卷積運算如圖所示，針對輸入資料，以固定的間隔，一邊移動，一邊套用濾鏡視窗。在各個位置乘上濾鏡的元素與對應輸入的元素，並計算總合(這個部分也稱為稽和運算)。再將結果儲存在對應輸出的位置。在全部的位置執行這格過程，可以得到積運算的輸出。<br><br>

在全連接的神經網路中，除了權重參數之外，還有偏權值。CNN的濾鏡參數對應的是前面提到的「權重」而且在CNN中，也有偏權值，下圖表式卷積運算套用濾鏡後的運算。包含偏權值的卷積運算處理流程。<br>
![7.2.2卷積運算的偏權值](./img/7.2.2.1.PNG)<br>
可以看到，偏權值隨時都只有一值，這個值要加在套用濾鏡後的所有元素。


## 7.2.3 填補(padding)
進行卷積處理之前，必須在輸入資料的周圍填上固定的資料(例如0)，這個動作稱作**填補( _padding_ )**，也是在卷積運算中，常用的處理。如圖所示，圖中對於大小為(4x4)的輸入資料，進行寬度為1的填補。<br>
![7.2.3padding](./img/7.2.3.PNG)<br><br>

{Note}<br>
使用填補的理由是為了調整輸出大小。假設在大小為(4x4)的輸入資料中，套用(3x3)的濾鏡時，輸出大小變成(2x2)，輸出資料只會縮小輸入資料的2個元素。如此一來，在反覆進行卷積運算的多層網路中，就會造成問題。如果每次進行卷積運算時，空間就會縮小的話，到了某個階段，輸出大小就會變成1，而無法再進行卷積運算，使用填補就是為了避免出現這樣的狀況。將填補大小設定為1，相對於輸入大小為(4x4)，輸出大小也會保持(4x4)。因此，藉由卷積運算，可以維持固定的空間大小，將資料傳遞給下一層。


### 7.2.4 步伐
套用濾鏡的位置間隔稱為**步伐( _stride_ )**，若我們把步伐變大，輸出大小就會變小，但我們把填補變大，輸出大小就會變大；所以，我們要把這種關係公式化。<br><br>

這裡假設輸入大小為$(H,W)$，濾鏡大小為$(FH,FW)$，輸出大小為$(OH,OW)$，填補為$P$，步伐為$S$，可以利用以下算式，計算輸出大小。<br>
$
OH = \frac{H+2P-FH}{S} + 1 \tag{7.1}
$

$
OW = \frac{W+2P-FW}{S} + 1 \tag{7.1}
$<br>
我們來練習一題：<br>
* 輸入大小：(28, 31)、填補：2、步伐：3、濾鏡大小：(5, 5)<br>

$
OH = \frac{28+(2*2)-5}{3} + 1 = 10
$<br>

$
OW = \frac{31+(2*2)-5}{3} + 1 = 11
$<br><br>

這裡必須注意的是，我們一定要讓算式(7.1)整除，假如輸出大小無法整除(結果為小數)，那麼我們就必須採取輸出錯誤的對策。


### 7.2.5 三維資料的卷積運算
三維資料包含，水平、垂直方向的形狀資料，以及色板方向所形成的三維資料。圖為三維資料的卷積運算的步驟。<br>
![三維資料的卷積運算範例](./img/7.2.5.PNG)<br>
這裡要注意的是，濾鏡大小可以隨意設定，但是色板數與輸入資料的色板數必須同值。這個範例的濾鏡大小是(3x3)，因此色板數量只能設定成3。


### 7.2.6 用區塊來思考
我們也可以將三維卷積運算中的資料或濾鏡想像成立體區塊，比較容易思考，例如，色板數C、高度H、寬度W的資料形狀會寫成(C, H, W )，濾鏡則由色板數C、濾鏡高度FH、寬度FW來表示(C, FH, FW)。<br>
![多個濾鏡的卷積運算](./img/7.2.6.jpg)<br>
由圖可知，套用FN個濾鏡，也會產生FN個輸出的特徵圖。<br><br>

在卷積運算中(和全連接層相同)，也有偏權值與權重，偏權值是每個色板只有一個資料。這裡偏權值形狀是(FN, 1, 1)，濾鏡輸出結果的形狀是(FN, OH, OW)。<br>
![卷積運算的處理流程(加上偏權值)](./img/7.2.6.1.jpg)


## 7.3 池化層
池化層是縮小垂直、水平空間的運算，如圖所示：<br>
![池化層運算](./img/7.3.PNG)<br>
其中，「最大池化」是用來取得每個視窗的最大值，而「平均池化」則是指，從每個視窗取出平均值。


### 7.3.1 池化層的特色
池化層有以下特色<br>
* 沒有學習參數<br>
&emsp; 池化層與卷積層不同，沒有學習參數。池化層指進行從目標區域取得最大值( 或平均值 )的處理，所以沒有必需學習的參數存在。<br>
* 色板數量不變<br>
&emsp; 輸入資料與輸出資料的色板數量不會隨著池化算而改變，各個色板進行獨立的運算。<br>
* 對微小位置變化很穩健<br>
&emsp; 即使輸入資料出現小偏差，池化仍會回傳相同結果(因為只取最大或平均池化)。因此，對輸入資料的微小偏差很穩健。


## 7.4 執行卷積層 / 池化層
或許我們直覺認為執行卷積層與池化層很複雜，但其實只要使用某個「妙招」，就可以輕鬆完成，而在執行的類別中，含有forwawrd與backward等方法(在第5章-誤差反向傳播法提及過)。


### 7.4.1 四維陣列
前面說明過，在CNN各層流動的是四維資料(FN, C, OH, OW)，使用Python執行，結果如下所示。


In [16]:
import numpy as np 
x = np.random.rand(10, 1, 28, 28) # 在此隨機初始化10個色板數為1，寬高為(28x28)的資料
print(x.shape)
print(x[0].shape)
print(x[9][0].shape) #.shape是由張量最前面的元素開始拆解，拆解到最後面的元素為止


(10, 1, 28, 28)
(1, 28, 28)
(28, 28)


### 「妙招」
這樣在CNN就會變成要處理四維資料，使得執行卷積運算的過程變得很複雜，其實只要使用下面要說明的im2col這個「妙招」，問題就會變得簡單。


### 7.4.2 利用im2col展開
執行卷積運算時，按部就班的處理，需要重疊多重for陳述式。可是這樣有點麻煩，而且在Numpy使用for陳述式，還有處理速度緩慢的缺點(在Numpy存取元素時，最好盡量別用for陳述式)。這裡不執行for陳述式，改用方便的im2col函數來進行簡單處理。<br><br>

im2col是可以針對套用濾鏡(權重)，輕鬆展開輸入資料的函數。如下圖所示，對三維的輸入資料套用im2col，就會轉換成二維陣列(正確來說，是將含有批次數的四維資料轉換成二維資料)。<br>
![7.4.2](./img/7.4.2.PNG)<br><br>

im2col可以針對套用濾鏡的位置，輕易展開輸入資料。具體而言，如下圖所示，對輸入資料套用濾鏡的區域(三維區塊)往水平方向展開成1行。im2col就是對套用了濾鏡的所有位置進行這種展開處理。<br>
![7.4.2.1](./img/7.4.2.1.PNG)<br><br>

另外，上圖以易讀性為前提，把步幅設定成較大的數值，避免套用濾鏡的區域重疊。實際上，進行卷積運算時，幾乎濾鏡區域都會重疊。套用濾鏡的區域重疊時，利用im2col展開，展開後的元素數量，會變得比原本區塊的元素數量還要多。因此，使用im2col時，通常會產生占用比較多記憶體的缺點。可是，利用電腦統一計算大型陣列，有很多好處。例如，計算陣列的函式庫(線性代數的函式庫)等，可以先將陣列計算最佳化，再快速執行大型陣列的乘法運算。因此，利用還原陣列計算的方法，能有效運用線性代數函式庫。<br><br>

{Note}<br>
im2col是「image to column」的縮寫，意思是將影像轉換成陣列。在Caffe及Chainer等深度學的框架中，皆含有名為im2col的函數，執行卷積層時，可以使用各個im2col來進行處理。<br><br>

利用im2col展開輸入資料，之後只要把卷積曾的濾鏡(權重)展開成1行，計算2個陣列的乘積(參考下圖)。這個部分與在全連接層Affine層進行的處理幾乎一模一樣。<br>
![7.4.2.2](./img/7.4.2.2.PNG)<br>
卷積運算的濾鏡處理詳細說明：往垂直方向把濾鏡展開成1行，計算以im2col展開的輸入資料與濾鏡矩陣(權重矩陣)的乘積，最後調整(reshape)輸出資料的大小。<br><br>

如上圖所示，利用im2col方法輸出的結果是二維陣列，CNN的資料型態是四維陣列，為求方便處理，將二維輸出資料調整成適當形狀，以上就是卷積層的執行流程。


### 7.4.3 執行卷積層
本書提供了im2col函數。請把這個im2col函數想像成在進行黑箱測試(別在意執行的內容)。此外，原始檔案位於common/util.py中，只有10行程式碼。<br><br>

im2col考量到「濾鏡大小」、「步幅」、「填補」，把輸入資料展開成二維陣列。接下來，我們來實際使用im2col。


In [1]:
import sys, os
import numpy as np
sys.path.append(os.pardir)
from dl_ex.common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) #im2col(input_data, filter_h, filter_w, stride=1, pad=0)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) #(9,75)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) #(90,75)


(4, 12)
(90, 75)


### 解說
這裡顯示了兩個範例。第一個批次大小為1，色板為3的7x7資料，第二個是批次大小為10，資料形狀和第1個相同的範例。分別套用im2col函數，這兩個範例，第二維的元素數量是75。這是濾鏡(色板3、大小5x5)的元素數量總和。此外，批次大小為1的時候，im2col的結果是大小為(9x75)。然而，第2個範例的批次大小為10，所以儲存了(9,75)的10倍資料(90x75)。而兩個範例的第一個維度是指，用了多少次數(多少視窗)走完其中一個權重矩陣。因此，若print(col1.shape)輸出結果為(9,75)，則代表有75個以9格視窗走完的輸出矩陣。<br><br>

接下來，要用im2col執行卷積層，這裡以名稱為Convolution的類別來執行卷積層。


In [2]:
class Convolution:
    def __init__(self, w, b, stride=1, pad=0):
        self.w = w
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.w.shape
        N, C, H, w = x.shape
        out_h = int(1 + 2*self.pad - FH / self.stride)
        out_w = int(1 + 2*self.pad - FW / self.stride)
        
        ## 重要部分開始 ##
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_w = self.w.reshape(FN, -1).T #展開濾鏡((將C,FH,FW)的N個濾鏡結合成一個矩陣)
        out = np.dot(col, col_w) + self.b # 與Affine層做的事情是一樣的(見7.4.2.2圖)
        ## 重要部分結束 ##
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) #改變序列(將原本的張量的(0,1,2,3)軸，改變成(0,3,1,2)軸)
               
        return out
          

### 解說
在執行卷積層過程，以#展開濾鏡那部分最為重要重要的部分。包含以im2col展開輸入資料，濾鏡也使用reshape展開成二維陣列，然後再計算展開後的矩陣乘積。<br><br>

展開濾鏡的位置，(如圖7.4.2.2)所示，將各濾鏡的區塊展開成1行。reshape(FN, -1)設定成-1，這是reshape的方便功能之一。<br><br>

接下來，我們要使進行卷積層的反向傳播，這個部分與Affine層的執行過程也有很多共通點，不過要注意到，要執行im2col的反向處理，要使用col2im的函數來因應這個問題。


